In [ ]:
import numpy as np
import matplotlib as mpl
from numpy import float32, float64
import matplotlib.pyplot as plt
from functools import partial
import crudepy
import itertools
from collections import namedtuple
from crudepy.numeric import integrate_trap
import math
import scipy as sp
from numba import njit
import matplotlib.animation as animation
from scipy.optimize import fsolve

In [ ]:
# For interactive plots
%matplotlib notebook

In [ ]:
label_font = 12
markersize = 8

In [ ]:
def setup_axes(ax: plt.Axes):
    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True)
    ax.grid(True, which="both")
    plt.rcParams.update({"figure.autolayout": True})

In [ ]:
def save_animation(x, y, name, interval=64):
    fig, ax = plt.subplots(1, 1, figsize=(11.7, 8.3))

    tau = 1.0
    line = ax.plot(x[0], y[0])[0]
    min_x = np.min(x)
    max_x = np.max(x)
    min_y = np.min(y)
    max_y = np.max(y)

    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)

    def update(frame):
        line.set_xdata(x[:frame])
        line.set_ydata(y[:frame])
        return line

    ani = animation.FuncAnimation(
        fig=fig, func=update, frames=len(x), interval=interval
    )
    plt.show()

    with open(name, "w") as file:
        print(ani.to_html5_video(), file=file)

## Задача 1

### New variables

$$
\begin{cases}
    x_1 = x \\
    x_2 = x' = x_1'\\
    x_3 = y \\
    x_4 = y' = x_3' \\
\end{cases}
$$

### System of first order differential equations

$$
\begin{cases}
    x_1' = x_2 \\
    x_2' = - \frac{x_1}{m L} T \\
    x_3' = x_4 \\
    x_4' = - \frac{x_3}{m L} T - \frac{F(t)}{m}
\end{cases}
$$

$$
x_1^2 + x_3^2 = L^2 \to x_1 x_1' + x_3 x_3' = 0 \to (x_1')^2 + (x_3')^2 + x_1 x_1'' + x_3 x_3'' = 0 \\
x_1 x_2' + x_3 x_4' = - [(x_1')^2 + (x_3')^2] = - [x_2^2 + x_4^2] \\
x_1 x_2' + x_3 x_4' = - \frac{L T + y F(t)}{m} \\
x_2^2 + x_4^2 = \frac{L T + y F(t)}{m} \\
L T = m (x_2^2 + x_4^2) - yF(t) \\
$$

In [ ]:
# https://perso.crans.org/besson/publis/notebooks/Runge-Kutta_methods_for_ODE_integration_in_Python.html
def rk4(fs: np.array, xs: np.array, t: float, step: float, **kwargs):
    def get_next_value(f, y):
        k_1 = f(xs, t, **kwargs)
        k_2 = f(xs + k_1 * step / 2.0, t + step / 2.0, **kwargs)
        k_3 = f(xs + k_2 * step / 2.0, t + step / 2.0, **kwargs)
        k_4 = f(xs + k_3 * step, t + step, **kwargs)
        return y + (step / 6.0) * (k_1 + 2.0 * k_2 + 2.0 * k_3 + k_4)

    return np.vectorize(get_next_value)(fs, xs)

In [ ]:
def calculate_tension(xs: np.array, t, m: float, f, l, *args, **kwargs) -> float:
    return (m * (xs[1] ** 2.0 + xs[3] ** 2.0) - xs[2] * f(t, m)) / l


def f_1(xs: np.array, *args, **kwargs) -> float:
    return xs[1]


def f_2(xs: np.array, t, m, l, f, *args, **kwargs) -> float:
    tension = calculate_tension(xs, t, m, f, l)
    return -1.0 * xs[0] / (m * l) * tension


def f_3(xs: np.array, *args, **kwargs) -> float:
    return xs[3]


def f_4(xs: np.array, t, m, l, f, *args, **kwargs) -> float:
    tension = calculate_tension(xs, t, m, f, l)
    return -1.0 * xs[2] / (m * l) * tension - f(t, m) / m

In [ ]:
def j_1(xs: np.array, *args, **kwargs) -> np.array:
    return np.array([0.0, 1.0, 0.0, 0.0])


# https://www.wolframalpha.com/input?i2d=true&i=Partial%5BDivide%5B-Subscript%5Bx%2C1%5D%2Cm+l%5DDivide%5Bm%2Cl%5D%5C%2840%29Power%5BSubscript%5Bx%2C2%5D%2C2%5D%2BPower%5BSubscript%5Bx%2C4%5D%2C2%5D%5C%2841%29+%2B+Subscript%5Bx%2C3%5DDivide%5BSubscript%5Bx%2C1%5D%2Cm+l%5D+f%5C%2840%29t%5C%2841%29%2CSubscript%5Bx%2C1%5D%5D
def j_2(xs: np.array, t, m, l, f, *args, **kwargs) -> np.array:
    return np.array(
        [
            (l * xs[2] * f(t, m) - m * xs[2] ** 2.0) / (l**2.0 * m)
            - xs[3] ** 2.0 / l**2.0,
            -2.0 * xs[0] * xs[1] / l**2.0,
            xs[0] * f(t, m) / (l * m),
            -2.0 * xs[0] * xs[3] / l**2.0,
        ]
    )


def j_3(xs: np.array, *args, **kwargs) -> np.array:
    return np.array([0.0, 0.0, 0.0, 1.0])


def j_4(xs: np.array, t, m, l, *args, **kwargs) -> np.array:
    return np.array(
        [
            0.0 - 2.0 * xs[1] * xs[2] / l**2.0,
            (2 * l * xs[2] * f(t, m) - m * (xs[1] ** 2.0 + xs[3] ** 2.0))
            / (l**2.0 * m),
            -2.0 * xs[2] * xs[3] / l**2.0,
        ]
    )

In [ ]:
m = 2.0  # kg
l = 8.0  # m
g = lambda t: 0.01 * np.cos(2 * np.pi * t)  # additional force
f = lambda t, m: m * (9.81 + g(t))  # gravity
fs = np.array([f_1, f_2, f_3, f_4])  # right-hand side
js = np.array([j_1, j_2, j_3, j_4])  # jacobians for right-hand side
xs_0 = np.array([3.0, 0.0, -np.sqrt(55.0), 0.0])  # initial position
step = 0.01  # seconds

In [ ]:
t = 0.0  # initial time
t_b = 4.0  # upper bound on time
xs = xs_0  # initial conditions
solution = []

while t <= t_b:
    xs = rk4(fs, xs, t, step, m=m, l=l, f=f)
    solution.append(xs)
    t += step

x = np.array(list(map(lambda y: y[0], solution)))
y = np.array(list(map(lambda y: y[2], solution)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11.7, 8.3))
ax.set_title("Рис. 1. Решение явной схемой RK4")
ax.plot(x, y)
fig.savefig("output/task1-explicit-rk4.pdf", transparent=False, bbox_inches="tight")

In [ ]:
save_animation(x, y, "output/task1-explicit-rk4.html")

In [ ]:
def newton_solve(x0, f, jacob, eps: float64, norm=np.inf, max_iter=1000000) -> np.array:
    cur = prev = x0

    for i in range(max_iter):
        j = jacob(*prev)
        inv_jacob = np.linalg.inv(j)
        delta = np.matmul(inv_jacob, f(*prev))
        cur -= delta
        err = np.linalg.norm(delta, norm)
        if err < eps:
            return cur

    raise RuntimeWarning("Simple iteration does not converge or is slowly converging")

In [ ]:
# Implementation of Explicit and Implicit Runge-Kutta methods to
# reduce the computational cost of pollutant transport modeling
# Ioannis Charis
# https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_method
def gl4(
    fs: np.array,
    js: np.array,
    xs: np.array,
    t: float,
    step: float,
    eps: float,
    max_iter: int = 10,
    **kwargs
):
    # Butcher table for Gauss–Legendre
    a11 = 0.25
    a12 = 0.25 - np.sqrt(3.0) / 6.0
    a21 = 0.25 + np.sqrt(3.0) / 6.0
    a22 = 0.25

    b1 = 0.5
    b2 = 0.5

    c1 = 0.5 - np.sqrt(3.0) / 6.0
    c2 = 0.5 + np.sqrt(3.0) / 6.0

    def get_next_value(f, j, y):
        k = f(xs, t, **kwargs)

        xs_1_guess = xs + c1 * step * k
        xs_2_guess = xs - c2 * step * k

        k_1_guess = f(xs_1_guess, t + c1 * step, **kwargs)
        k_2_guess = f(xs_2_guess, t + c2 * step, **kwargs)
        ks_0 = np.array([k_1_guess, k_2_guess])

        def f_for_k(k_1_cur, k_2_cur):
            return step * np.array(
                [
                    k_1_cur
                    - f(
                        xs + (k_1_cur * a11 + k_2_cur * a12) * step,
                        t + c1 * step,
                        **kwargs
                    ),
                    k_2_cur
                    - f(
                        xs + (k_1_cur * a21 + k_2_cur * a22) * step,
                        t + c2 * step,
                        **kwargs
                    ),
                ]
            )

        def j_for_k(k_1_cur, k_2_cur):
            j1 = j(xs + (k_1_cur * a11 + k_2_cur * a12) * step, t + c1 * step, **kwargs)
            j2 = j(xs + (k_1_cur * a12 + k_2_cur * a22) * step, t + c2 * step, **kwargs)

            return np.eye(2) - step * np.array(
                [
                    [a11 * np.sum(j1), a12 * np.sum(j1)],
                    [a21 * np.sum(j2), a22 * np.sum(j2)],
                ]
            )

        ks = newton_solve(ks_0, f_for_k, j_for_k, eps=eps, max_iter=max_iter)
        k_1, k_2 = ks
        return y + step * (b1 * k_1 + b2 * k_2)

    return np.vectorize(get_next_value)(fs, js, xs)

In [ ]:
step = 0.01  # seconds

In [ ]:
t = 0.0  # initial time
t_b = 4.0  # upper bound on time
xs = xs_0  # initial conditions
solution = []

while t <= t_b:
    xs = gl4(fs, js, xs, t, step, 1e-6, 1000, m=m, l=l, f=f)
    solution.append(xs)
    t += step

x = np.array(list(map(lambda y: y[0], solution)))
y = np.array(list(map(lambda y: y[2], solution)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11.7, 8.3))
ax.set_title("Рис. 2. Решение неявной схемой GL4")
ax.plot(x, y)
fig.savefig("output/task1-implicit-gl4.pdf", transparent=False, bbox_inches="tight")

In [ ]:
save_animation(x, y, "output/task1-implicit-gl4.html")

## Задача 2

## Задача 3

### New variables

$$
\begin{cases}
    u = y \\
    v = y' \\
\end{cases}
$$

### System of first order differential equations

$$
\begin{cases}
    u' = v \\
    v' = (3 - t^2) (cos(t) + v) + 2 - 6 t + 2 t^3 + (t^2 - 3) e^t sin(t) (1 + cos(t)) + cos(t) (e^t + (t^2 - 1) + t^4 - 3t^2)
\end{cases}
$$

$$
\begin{cases}
    u(0) = 0 \\
    u(\pi) = \pi^2
\end{cases}
$$

In [ ]:
u_0 = 0.0

In [ ]:
@njit
def f(xs, t):
    u, v = xs
    return v

In [ ]:
@njit
def g(xs, t):
    u, v = xs
    return (
        (3.0 - t**2.0) * (np.cos(t) + v)
        + 2.0
        - 6.0 * t
        + 2.0 * t**3.0
        + (t**2.0 - 3.0) * np.exp(t) * np.sin(t) * (1.0 + np.cos(t))
        + np.cos(t) * (np.exp(t) + (t**2.0 - 1.0) + t**4.0 - 3.0 * t**2.0)
    )

In [ ]:
@njit
def get_next_value_rk4(f, y: float, xs: np.array, t: float, step: float):
    k_1 = f(xs, t)
    k_2 = f(xs + k_1 * step / 2.0, t + step / 2.0)
    k_3 = f(xs + k_2 * step / 2.0, t + step / 2.0)
    k_4 = f(xs + k_3 * step, t + step)

    return y + (step / 6.0) * (k_1 + 2.0 * k_2 + 2.0 * k_3 + k_4)

In [ ]:
# https://perso.crans.org/besson/publis/notebooks/Runge-Kutta_methods_for_ODE_integration_in_Python.html
@njit
def rk4(f, x: float, xs: np.array, t: float, step: float):
    return get_next_value_rk4(f, x, xs, t, step)

In [ ]:
def solve_initial_value_problem(xs_0: np.array, step: float = 0.001):
    global f
    global g

    t = 0.0  # initial time
    t_b = np.pi  # upper bound on time
    xs = xs_0  # initial conditions
    solution = []

    while t <= t_b:
        u, v = xs
        xs = np.array([rk4(f, u, xs, t, step), rk4(g, v, xs, t, step)])
        solution.append(np.array([*xs, t]))
        t += step

    return np.array(solution).transpose()

In [ ]:
# https://pythonnumericalmethods.berkeley.edu/notebooks/chapter23.02-The-Shooting-Method.html
def objective(v_0, step: float = 0.001):
    us, vs, _ = solve_initial_value_problem(np.array([u_0, *v_0]), step)
    expected_u = np.pi**2.0
    return us[-1] - expected_u

In [ ]:
(v_0,) = fsolve(objective, 0.0)
us, vs, ts = solve_initial_value_problem(np.array([u_0, v_0]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11.7, 8.3))
ax.set_title("Рис. 3. Решение краевой задачи методом стрельбы")
ax.plot(ts, us)
ax.grid()
fig.savefig("output/task3-shooting.pdf", transparent=False, bbox_inches="tight")